In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from factor_design_distributed_ver1 import utils, factor_design, factor_backtest
import pandas as pd
import numpy as np
from tqdm import tqdm

# load all relevant resources 

In [16]:
# play around the data 
df_train = pd.read_csv('./research_train_set.csv')

# We built a index map to speed up the factor design, you don't need to worry about this.
col2index_map = utils.load_json('./factor_design_distributed_ver1/col2index_map.json')
# df_train in a dictionary format
df_train_dic_sorted = utils.load_json('./factor_design_distributed_ver1/df_train_dic_sorted.json')

In [19]:
existed_factors = utils.add_factor_to_existed() # this function will add all temp files to the existed factor, then return the existed factor. usually take 1 min

matching files: []
No temporary files found, no new factors added, returning existed_factors


In [20]:
existed_factors.keys()

dict_keys(['origin_seconds_in_bucket', 'origin_imbalance_size', 'origin_imbalance_buy_sell_flag', 'origin_reference_price', 'origin_matched_size', 'origin_far_price', 'origin_near_price', 'origin_bid_price', 'origin_bid_size', 'origin_ask_price', 'origin_ask_size', 'origin_wap', 's1_imbalance_signed_pow_opt', 'stock_20s_ret_deviate_from_index', 'stock_40s_ret_deviate_from_index_rank_hedged_opt'])

In [5]:
factor_performance = utils.load_json('./factor_design_distributed_ver1/factor_performance.json')

# Factor Design

In [6]:
# sample factor
def s1_imbalance(current_data: dict, hist_list=[]) -> np.ndarray:
    '''
    This will be the main function to design your factors for the competition. Please
    define only one factor here each time. We provide you with:

    Current_data: a dictionary in the format of {column_name: column_value}, where column_name is from the original
    dataframe

    hist_list: A list for you to save the previous factor values (optional). For instance,
    if you are calculating a 100-day Moving Average (MA), then you can save the first calculated
    MA in hist_list, and then for the next MA calculation, you can use the saved ones.
    '''
    ###################### ADD YOUR CODE HERE FOR FACTORS DESIGN ######################
    # convert the current_data to your choice of numpy or pandas dataframe
    # current_data = pd.DataFrame(current_data)
    current_data = np.array(list(current_data.values()),
                            dtype=float).T  # this is faster than pd.DataFrame(current_data).values
    res = (current_data[:, col2index_map['bid_size']] - current_data[:, col2index_map['ask_size']]) / \
          (current_data[:, col2index_map['bid_size']] + current_data[:, col2index_map['ask_size']])
    ############################## NAN/Inf handling ######################################
    # if you have nan in your factor value, please fill it reasonably
    # res = np.nan_to_num(res) # this is slow because it also checks for inf.
    # res = np.where(np.isnan(res), 0, res)  # this is slightly faster than np.nan_to_num
    res[np.isnan(res)] = 0  # this is the fastest way to fill nan
    ############################## END OF YOUR CODE ##############################
    return -res  # The return value MUSE BE a numpy array, with no NaN value
    ####################################################################################


In [7]:
new_factor_list = ['s1_imbalance']

In [8]:
# DO NOT MODIFY THE FOLLOWING CODE
# Run this cell once you want to calculate your factor values and prepare for the test of your factor performance!
new_factors = {
    factor_name: utils.flatten_factor_value(
        factor_design.run_factor_value(df_train_dic_sorted, eval(factor_name), factor_name), factor_name)[factor_name]
    for factor_name in tqdm(new_factor_list)}

  0%|          | 0/1 [00:00<?, ?it/s]

Start calculating factor s1_imbalance
Finished calculating factor s1_imbalance for 0 dates


C:\Users\kevin\AppData\Local\Temp\ipykernel_14860\1115659932.py:19: RuntimeWarning: invalid value encountered in divide
  res = (current_data[:, col2index_map['bid_size']] - current_data[:, col2index_map['ask_size']]) / \


Finished calculating factor s1_imbalance for 100 dates
Finished calculating factor s1_imbalance for 200 dates
Finished calculating factor s1_imbalance for 300 dates
Finished calculating factor s1_imbalance for 400 dates
Accepted!!: Used 6.03 seconds for calculation factors. The limit is 300 seconds.


100%|██████████| 1/1 [00:06<00:00,  6.52s/it]


In [9]:
# check if all values in new_factor do not contain nan, no NA is allowed in the factor value
for factor_name, factor_value in new_factors.items():
    assert not np.isnan(factor_value).any(), f'{factor_name} contains nan'
    assert not np.isinf(factor_value).any(), f'{factor_name} contains inf'
    print(f'{factor_name} has correlation to target {np.corrcoef(factor_value, df_train["target"].fillna(0))[0, 1]}')


s1_imbalance has correlation to target 0.11723116519487425


In [10]:
# simple check for pairwise correlation
for factor_name, factor_value in new_factors.items():
    for existed_factor_name, existed_factor_value in existed_factors.items():
        correlation = np.corrcoef(factor_value, existed_factor_value)[0, 1]
        if correlation > 0.4:# if correlation with existing factor < 0.4, there is no penalty for this factor
            print(f'{factor_name} has correlation to {existed_factor_name} {np.corrcoef(factor_value, existed_factor_value)[0, 1]}')
            print(f'    This existing factor have a correlation to target {np.corrcoef(existed_factor_value, df_train["target"].fillna(0))[0, 1]}')
            break
print('Other factor has correlation to target < 0.4')

s1_imbalance has correlation to s1_imbalance_signed_pow_opt 0.992082905345458
    This existing factor have a correlation to target 0.11812815373942646
Other factor has correlation to target < 0.4


# Factor Backtesting

In [11]:
demo_backtest = factor_backtest.Factor_Backtest(
    existed_factors=existed_factors,
    testing_factors=new_factors,
    factor_performance=factor_performance
)

In [12]:
demo_backtest.run_testing()


Start testing factor: s1_imbalance..........

_______________________________________Factor Performance_______________________________________________________
Factor s1_imbalance PASSED in-sample performance check with correlation coefficient 0.11723116519487425

___________________________________Factor pairwise Correlation__________________________________________________
Factor s1_imbalance FAILED in-sample correlation check with factor s1_imbalance_signed_pow_opt, 
The pair have correlation 0.992082905345458
This falls in type 2 threshold, >0.7

The required performance is 1.2 times the old factor performance 0.11812815373942646 = 0.14175378448731174.
Performance of the new factor now is  0.11723116519487425

_______________________________________Factor Conclusion_______________________________________________________
Factor s1_imbalance FAILED


______________________________________Overall Conclusion________________________________________________________
Factors passed all the